# Numpy高级应用

In [4]:
import numpy as np

## ndarray对象内部机理

### ndarray内部组成结构

NumPy的ndarray提供了一种将**同质数据块**(可以是连续或跨越的,稍后将详细讲解)解释为**多维数组**对象的方式。正如你之前所
看到的那样,数据类型(dtype)决定了数据的解释方式,比如浮点数、整数、布尔值等。

ndarray如此强大的部分原因是所有数组对象都是数据块的一个跨度视图(strided view)。你可能想知道数组视图arr[::2,::-1]不复制任
何数据的原因是什么。简单地说,ndarray不只是一块内存和一个dtype,它还有跨度信息,这使得数组能以各种步幅(step size)在内存中
移动。更准确地讲,ndarray内部由以下内容组成：
* 一个指向数组(一个系统内存块)的**指针**。
* **数据类型**或dtype。
* 一个表示数组**形状**(shape)的**元组**,例如,一个10×5的数组,其形状为(10,5)。
        np.ones((10, 5)).shape
        (10, 5)
* 一个**跨度元组**(stride),其中的整数指的是为了前进到当前维度下一个元素需要“跨过”的字节数,例如,一个典型的(C顺序,稍后将详细讲解)3×4×5的float64(8个字节)数组,其跨度为(160,40,8)。
        np.ones((3, 4, 5),dtype=np.float64).strides
        (160, 40, 8)
        
虽然NumPy用户很少会对数组的跨度信息感兴趣,但它们却是构建**非复制式数组视图**的重要因素。跨度甚至可以是**负数**,这样会使
数组在内存中**后向**移动,比如在切片obj[::-1]或obj[:,::-1]中就是这样的，下图很好的描述了ndarray的内部结构组成：

![ndarray的内部结构](https://raw.githubusercontent.com/NemoHoHaloAi/machine_learning/master/python%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/images/ndarray%E7%9A%84%E5%86%85%E9%83%A8%E7%BB%93%E6%9E%84.png)

### NumPy数据类型体系

你可能偶尔需要检查数组中所包含的是否是整数、浮点数、字符
串或Python对象。因为浮点数的种类很多,判断dtype是否属于某个大
类的工作非常繁琐。幸运的是,dtype都有一个**超类**(比如np.integer和
np.floating),它们可以跟**np.issubdtype**函数结合使用:

        ints = np.ones(10, dtype=np.uint16)
        floats = np.ones(10, dtype=np.float32)
        np.issubdtype(ints.dtype, np.integer)
        True
        np.issubdtype(floats.dtype, np.floating)
        True
        
调用dtype的**mro**方法即可查看其所有的父类:
        
        np.float64.mro()
        [numpy.float64,
        numpy.floating,
        numpy.inexact,
        numpy.number,
        numpy.generic,float,
        object]
        
大部分NumPy用户完全不需要了解这些知识,但是这些知识偶尔还是能派上用场的。下图说明了dtype体系以及父子类关系：
![dtype类体系结构](https://github.com/NemoHoHaloAi/machine_learning/blob/master/python%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/images/dtype%E7%B1%BB%E4%BD%93%E7%B3%BB%E7%BB%93%E6%9E%84.png?raw=true)

## 高级数组操作

除**花式索引**、**切片**、**布尔条件取子集**等操作之外,数组的操作方
式还有很多。虽然pandas中的高级函数可以处理数据分析工作中的许
多重型任务,但有时你还是需要编写一些在现有库中找不到的数据算
法。

### 数组重塑

鉴于我们已经学过的有关NumPy数组的知识,当你知道“无需复
制任何数据,数组就能从一个形状转换为另一个形状”时应该会感到
有一点吃惊。只需向数组的实例方法reshape传入一个表示新形状的元
组即可实现该目的。

#### 一维到多维 -- reshape

In [5]:
arr = np.random.randn(8)
arr

array([ 2.19968261, -2.51749262, -2.04008776, -0.60886891,  1.78442242,
        0.90907107,  0.81324603,  0.60564975])

In [6]:
arr.reshape(4,2)

array([[ 2.19968261, -2.51749262],
       [-2.04008776, -0.60886891],
       [ 1.78442242,  0.90907107],
       [ 0.81324603,  0.60564975]])

In [7]:
(arr.reshape(4,2)).reshape(2,4)

array([[ 2.19968261, -2.51749262, -2.04008776, -0.60886891],
       [ 1.78442242,  0.90907107,  0.81324603,  0.60564975]])

In [8]:
arr.reshape(np.random.randn(2,4).shape)

array([[ 2.19968261, -2.51749262, -2.04008776, -0.60886891],
       [ 1.78442242,  0.90907107,  0.81324603,  0.60564975]])

In [10]:
arr.reshape(2,-1) # -1,它表示该维度的大小由数据本身推断而来

array([[ 2.19968261, -2.51749262, -2.04008776, -0.60886891],
       [ 1.78442242,  0.90907107,  0.81324603,  0.60564975]])

#### 多维到一维 -- ravel,flatten

In [11]:
arr.reshape(2,4).ravel()

array([ 2.19968261, -2.51749262, -2.04008776, -0.60886891,  1.78442242,
        0.90907107,  0.81324603,  0.60564975])

In [12]:
arr.reshape(2,4).flatten()

array([ 2.19968261, -2.51749262, -2.04008776, -0.60886891,  1.78442242,
        0.90907107,  0.81324603,  0.60564975])

### C和Fortran顺序

与其他科学计算环境相反(如R和MATLAB),NumPy允许你更为灵活地控制数据在内存中的布局。默认情况下,NumPy数组是按行优先顺序创建的。在空间方面,这就意味着,对于一个二维数组,**每行**中的数据项是被存放在**相邻内存位置**上的。另一种顺序是列优先顺序,它意味着(猜到了吧)每列中的数据项是被存放在相邻内存位置上的。

由于一些历史原因,行和列优先顺序又分别称为C和Fortran顺序。在FORTRAN 77中(前辈们的语言),矩阵全都是列优先的。

像reshape和reval这样的函数,都可以接受一个表示数组数据存放顺序的**order**参数。一般可以是'**C**'或'**F**'(还有'A'和'K'等不常用的选项,具体请参考NumPy的文档)。

In [16]:
arr.reshape(2,4)

array([[ 2.19968261, -2.51749262, -2.04008776, -0.60886891],
       [ 1.78442242,  0.90907107,  0.81324603,  0.60564975]])

In [14]:
arr.reshape(2,4).ravel(order='C')

array([ 2.19968261, -2.51749262, -2.04008776, -0.60886891,  1.78442242,
        0.90907107,  0.81324603,  0.60564975])

In [15]:
arr.reshape(2,4).ravel(order='F')

array([ 2.19968261,  1.78442242, -2.51749262,  0.90907107, -2.04008776,
        0.81324603, -0.60886891,  0.60564975])

## 广播

## ufunc高级应用

## 结构化和记录式数组

## 更多有关排序的话题

## Numpy的Matrix类

## 高级数组输入输出

## 性能建议